# Text Mining Group Project


### Imports

In [28]:
#Don't forget to 'pip install tranformers, pip install torch' in conda prompt
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Data Acquisition

In [29]:
import json
#https://eightportions.com/datasets/Recipes/#fn:1 where the data is from
#scraped from 
#Foodnetwork.com, Epicurious.com, Allrecipes.com by someone

allr = open('recipes_raw_nosource_ar.json')
epi = open('recipes_raw_nosource_epi.json')
food = open('recipes_raw_nosource_fn.json')

data1 = json.load(allr) #data 1 has a lot of the word ADVERTISEMENT
data2 = json.load(epi) #data 2 looks good
data3 = json.load(food) #this too

#this load module loads the data into a dictionary 

for key, value in data3.items():
    print(value)
#example of a specific recipe with a certain key and the value is also a dictionary
print(data3["p3pKOD6jIHEcjf20CCXohP8uqkG5dGi"])

#example of only instructions
print(data3["p3pKOD6jIHEcjf20CCXohP8uqkG5dGi"]['instructions'])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




{'instructions': 'Preheat 10-inch Dutch oven on open fire. Meanwhile, soak chips in 1/2 cup of milk. This will be your bottom crust. Soak biscuits with remaining ingredients, add extra milk if needed for a wet consistency. Salt and pepper, to your taste. Remove oven from the fire, put chips in first then add the biscuit mixture. Return Dutch oven to the fire. Be careful when putting coals on top and bottom. Cook for 20 to 30 minutes or until slightly firm. Serve with cactus sauce.\nPuree all ingredients together and refrigerate.', 'ingredients': ['2 cups broken blue tortilla chips', '1 1/2 cups milk', '5 cups day old biscuits', '18 eggs', '1/2 cup chopped red bell peppers', '2 1/2 cups Mexican white cheese, shredded', 'Cactus Sauce, recipe follows', '14 ounces diced chiles', '3 to 4 ripe avocados'], 'title': 'Breakfast Casserole', 'picture_link': 'DnBsNqHvI7qTEdt3ANqGnqJV/i694EO'}
{'instructions': 'With a citrus reamer or your thumb, push a little cavity into the center of the lime to

# Data Organization

For this part, we really have to look at getting all of the recipes into the same format: so the model will learn to recognize the format, and start to use it

In [30]:
pass

# Language Recognition

With some help from: [TowardsDataScience](https://towardsdatascience.com/text-generation-with-python-and-gpt-2-1fecbff1635b)

### Sequence
Here we define an arbitrary recipe string sequence to initially train and test our model with. This text will be improved when we have acquired some data, but this will do for initial development:

In [131]:
sequence = """Ingredients: 3 tomatoes, garlic, spaghetti, Spanish chili's, cheese. Boil the pasta, then rinse. Cut chili's and mince garlic, add to pan
Add tomatoes and the pasta, and grate a generous amount of cheese
Serve with a touch of basil and a glass of good red wine
           """

seqdata = []
print(len(data3.keys()))
count = 0
for key, value in data3.items():
    check = False
    
    #check whether it has the key ingredients
    if "ingredients" in value.keys():
        check = True
    
    #amount of recipes you want
    if count <= 800 and check:
        count = count +1
        listingr = "ingredients: "
        for i in value["ingredients"]:
            listingr = listingr + i + " "
        if type(value["instructions"]) == str:
            r = listingr + value["instructions"]
            seqdata.append(r)
            continue

print(type(seqdata))
#first two recipes
print(seqdata[:2])

60039
<class 'list'>
['ingredients: 1/2 cup celery, finely chopped 1 small green pepper finely chopped 1/2 cup finely sliced green onions 1/4 cup chopped parsley 1 pound crabmeat 1 1/4 cups coarsely crushed cracker crumbs 1/2 teaspoon salt 3/4 teaspoons dry mustard Dash hot sauce 1/4 cup heavy cream 1/2 cup melted butter Toss ingredients lightly and spoon into a buttered baking dish. Top with additional crushed cracker crumbs, and brush with melted butter. Bake in a preheated at 350 degrees oven for 25 to 30 minutes or until delicately browned.', "ingredients: 2 pounds skirt steak, cut into 1/2-inch dice Kosher salt and fresh cracked black pepper 4 to 6 slices thick-cut applewood smoked bacon, 1/4-inch diced (about 1 cup) 1 1/2 cups 1/4-inch diced red onion 1/2 cup seeded and finely diced jalapenos (2 medium) 3 tablespoons minced garlic 1 teaspoon kosher salt 2 teaspoons fresh cracked black pepper Apple cider vinegar, best quality 1 (15-ounce) can cannellini beans, with liquid 1 (15-ou

In [123]:
#We use the pretrained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

GPT2's development can be found at: Radford, A., Wu, J., Child, R., Luan, D., Amodei, D., & Sutskever, I. (2018). Language Models are Unsupervised Multitask Learners. 24.

In [127]:
#Tokenizing the inputs
#print(seqdata[1])
#inputs = tokenizer.encode(sequence, return_tensors='pt')
#print(inputs)

tokenizer.pad_token = "tokenizer.eos_token"
#padding and truncation to make all tensors same size.
input_ids = tokenizer(seqdata, return_tensors='pt', padding=True,truncation = True).input_ids

print(input_ids) #it is weird that the tensor looks like this and is the same in every row

tensor([[  278, 23320,    25,  ..., 50256, 50256, 50256],
        [  278, 23320,    25,  ..., 50256, 50256, 50256],
        [  278, 23320,    25,  ..., 50256, 50256, 50256],
        ...,
        [  278, 23320,    25,  ..., 50256, 50256, 50256],
        [  278, 23320,    25,  ..., 50256, 50256, 50256],
        [  278, 23320,    25,  ..., 50256, 50256, 50256]])


# Language Generation

Here we generate a recipe based on the encoded cooking texts that we fed the model above. We will use the model.generate() function to do this.
Model.generate() has a lot of different variable options worth looking at, which can be used to optimize our model. For instance, the temperature variable can be tweaked between 0-5 for increased randomness, we can try no_repeat_ngram_size=2 to prevent repitition, or tweak top_k. We'll just have to play around a bit and see what works best

In [116]:
# We set the output length to 500 tokens
#EncodedRecipe = model.generate(inputs, max_length=500, do_sample=True)
#print(EncodedRecipe)
# We set the output length to 500 tokens
EncodedRecipe2 = model.generate(input_ids, max_length=1024, do_sample=True)
print(EncodedRecipe2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 1024, but ``max_length`` is set to 1024.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.


tensor([[  278, 23320,    25,  ..., 50256, 50256, 50256],
        [  278, 23320,    25,  ..., 50256, 50256, 50256],
        [  278, 23320,    25,  ..., 50256, 50256, 50256],
        ...,
        [  278, 23320,    25,  ..., 50256, 50256, 50256],
        [  278, 23320,    25,  ..., 50256, 50256, 50256],
        [  278, 23320,    25,  ..., 50256, 50256, 50256]])


In [132]:
#Decode the tokenized outputs
#recipe = tokenizer.decode(EncodedRecipe[0], skip_special_tokens=True)
#We filter out the initial context sequence given to the model (258 characters)
#print(recipe[259:])

#Decode the tokenized outputs
recipe2 = tokenizer.decode(EncodedRecipe2[0], skip_special_tokens=True)
#We filter out the initial context sequence given to the model (258 characters)
print(len(recipe2))
print(recipe2) #it does not really work yet

527
ingredients: 1/2 cup celery, finely chopped 1 small green pepper finely chopped 1/2 cup finely sliced green onions 1/4 cup chopped parsley 1 pound crabmeat 1 1/4 cups coarsely crushed cracker crumbs 1/2 teaspoon salt 3/4 teaspoons dry mustard Dash hot sauce 1/4 cup heavy cream 1/2 cup melted butter Toss ingredients lightly and spoon into a buttered baking dish. Top with additional crushed cracker crumbs, and brush with melted butter. Bake in a preheated at 350 degrees oven for 25 to 30 minutes or until delicately browned.


this repetition in this recipe is a great example of why we may have to try ways to prevent repetition in the model generation

## Saving Recipes

In [51]:
#Defining a function to save generated recipes as .txt files:
def save(recipe, filename):
    text_file = open(filename, "w", encoding = 'utf8')
    n = text_file.write(recipe)
    text_file.close()

In [52]:
#Saving the previously generated recipe:
save(recipe[259:], 'FirstRecipe.txt')
#Note: I am leaving out the 259 context characters

# Reinforcement learning
I had some inspiration for a potentially novel approach to take this model in: reinforcement learning. While the current model already performs relatively well on creating sensical recipe texts, the model still has absolutely no idea of 'taste', and which ingredients work well together and which don't. Therefore, it may be desireable to give the model some sort of loss function score on its recipes so it knows which recipes are good and which are bad, so the model can improve.
We could do this by either reading the recipes and rating them on how good we think they might be, or maybe even try to cook some of the things our model suggests and see how well they work, so we can help the model improve. While this may take some time and make the coding harder, it could be really cool to have a recipe generation model trained and optimized on actual real-world cooking, and I can hardly imagine anyone has ever done something like that before. Who knows, maybe the model will actually end up becoming a really good cook

Contributors: Emilia, Sergio, Tim